# Building Fast Queries on a CSV

Optimizing an algorithm's performance is a useful endeavor.

## Goal:

In this project, I will demonstrate the use of data preprocessing and understanding time complexity.

I will assume that I am working for a company that sells laptops, and I will perform the following tasks:

### Tasks:

* Retrieve a laptop from an inventory list using only it's ID Number.
* Determine if a promotional offer equals the cost of one or two laptops in the inventory.
* Determine the most expensive laptop a customer can afford, given a budget of $D$ dollars.

I will use [this laptop dataset here](https://www.kaggle.com/ionaskel/laptop-prices) to complete these tasks.

## Introducing the Data

In [1]:
import csv
import chardet
import time
import random

In [2]:
# Checking the encoding

with open('laptops.csv', mode ='rb') as file:
    raw_bytes = file.read()

chardet.detect(raw_bytes)

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}

The encoding is not UTF.

In [3]:
# Opening the file
with open('laptops.csv', encoding='ISO-8859-1') as file: 
    laptops = list(csv.reader(file))

In [4]:
# Viewing the first row
laptops[0]

['',
 'Company',
 'Product',
 'TypeName',
 'Inches',
 'ScreenResolution',
 'Cpu',
 'Ram',
 'Memory',
 'Gpu',
 'OpSys',
 'Weight',
 'Price_euros']

In [5]:
# Viewing the next five rows
laptops[1:6]

[['1',
  'Apple',
  'MacBook Pro',
  'Ultrabook',
  '13.3',
  'IPS Panel Retina Display 2560x1600',
  'Intel Core i5 2.3GHz',
  '8GB',
  '128GB SSD',
  'Intel Iris Plus Graphics 640',
  'macOS',
  '1.37kg',
  '1339.69'],
 ['2',
  'Apple',
  'Macbook Air',
  'Ultrabook',
  '13.3',
  '1440x900',
  'Intel Core i5 1.8GHz',
  '8GB',
  '128GB Flash Storage',
  'Intel HD Graphics 6000',
  'macOS',
  '1.34kg',
  '898.94'],
 ['3',
  'HP',
  '250 G6',
  'Notebook',
  '15.6',
  'Full HD 1920x1080',
  'Intel Core i5 7200U 2.5GHz',
  '8GB',
  '256GB SSD',
  'Intel HD Graphics 620',
  'No OS',
  '1.86kg',
  '575.00'],
 ['4',
  'Apple',
  'MacBook Pro',
  'Ultrabook',
  '15.4',
  'IPS Panel Retina Display 2880x1800',
  'Intel Core i7 2.7GHz',
  '16GB',
  '512GB SSD',
  'AMD Radeon Pro 455',
  'macOS',
  '1.83kg',
  '2537.45'],
 ['5',
  'Apple',
  'MacBook Pro',
  'Ultrabook',
  '13.3',
  'IPS Panel Retina Display 2560x1600',
  'Intel Core i5 3.1GHz',
  '8GB',
  '256GB SSD',
  'Intel Iris Plus Graphic

In [6]:
# Viewing the type of each variable
for val in laptops[1]:
    print(type(val))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


I will need to preprocess the data in order to answer the questions.

## Implementing the `Inventory()` Class

Instead of preprocessing the data now, I will instead preprocess it as a class object.

I will define a class `Inventory()`, make all preprocessing changes within this class, and define all attributes and methods that I will use to answer the questions and complete the tasks.

In [7]:
class Inventory():
    """
    A class of Inventory data
    
    ...
        
    Attributes
    ----------
    header : list
        The names of the columns
        
    rows : list
        The list of laptops and their features
        
    id_to_row : dict
        A dictionary with key : id, value : laptop row
        
    prices : set
        The set of all unique laptop prices
    
    rows_by_price : list
        The list of laptops and their features, ordered by price in ascending order
    
        
    Methods
    -------
    __init__ (csv_filename)
        Opens the csv file and preprocesses the data
        
    get_laptop_from_id (laptop_id)
        Returns laptop from given id value
        
    get_laptop_from_id_fast (laptop_id)
        Returns laptop from givein id value faster
        
    check_promotion_dollars (dollars)
        Returns boolean indicating if "dollars" equals the price 
        of any 1 laptop or the sum of the prices of any 2 laptops
        
    check_promotion_dollars_fast (dollars)
        Returns same boolean as mentioned above, but faster
        
    find_first_laptop_more_expensive (budget)
        Binary search algorithm. Returns the most expensive laptop equal to or below a given price
    """
    
    def __init__(self, csv_filename):
        """Opens the inventory file.
        
        Preprocesses the data:
        - Separates the header from the data rows
        - Names the `id` column
        - Makes the `price` and `id` columns integer-type
        - Creates an id:row dictionary
        - Creates a set of unique `prices`
        - Sorts the rows by `price`.
        
        Parameters
        ----------
        csv_filename : str
            The name of the csv inventory file
        """
        # Opening the file with the proper encoding
        with open(csv_filename, mode ='rb') as file: 
            raw_bytes = file.read()
        
        chardet_ = chardet.detect(raw_bytes)
        encoding = chardet_['encoding'] 
        
        with open(csv_filename, encoding=encoding) as file: 
            rows = list(csv.reader(file)) 
        
        # Separating the header from the data rows
        self.header = rows[0]
        self.rows = rows[1:]
        
        # Naming the `id` and `price` column
        self.header[0] = 'id'
        self.header[-1] = 'price'
        
        # Making the `id` and `price` columns integer types
        for row in self.rows:
            row[0] = int(row[0])
            row[-1] = int(float(row[-1]))
            
        # Creating the `id:row` dictionary and the `prices` set
        self.id_to_row = {}
        self.prices = set()
            
        for row in self.rows:
            self.id_to_row[row[0]] = row
            self.prices.add(row[-1])
        
        # Sorting the rows by `price`
        self.rows_by_price = sorted(self.rows, key = lambda row: row[-1])
    
    # Return laptop and its features, given the laptop's id
    def get_laptop_from_id(self, laptop_id):
        """Retrieves a laptop and its features given a laptop id
        
        Parameters
        ----------
        laptop_id : int
            The id of the laptop
        
        Returns
        -------
            list
        """
        for row in self.rows:    
            if row[0] == laptop_id:
                return row
        return None
    
    # Performs the same task as the above, but faster, using a dictionary
    def get_laptop_from_id_fast(self, laptop_id):
        """Retrieves a laptop and its features given a laptop id
        
        Parameters
        ----------
        laptop_id : int
            The id of the laptop
        
        Returns
        -------
            list
        """
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
        
    # Determine if a dollar amount equals price of any 1 laptop
    # or the sum of the prices of any two laptops
    def check_promotion_dollars(self, dollars):
        """Determines if a promotional dollar amount equals 
        the price of 1 laptop or the sum of the price of 2 laptops
        
        Parameters
        ----------
        dollars : int
            The dollar amount for the promotional offer
        
        Returns
        -------
            bool
        """
        for row in self.rows:  
            if row[-1] == int(dollars):
                return True
            for other_row in self.rows:
                if other_row[-1] + row[-1] == int(dollars):
                    return True
        return False
    
    # Performs the same task as the above, but faster, using sets
    def check_promotion_dollars_fast(self, dollars):
        """Determines if a promotional dollar amount equals 
        the price of 1 laptop or the sum of the price of 2 laptops
        
        Parameters
        ----------
        dollars : int
            The dollar amount for the promotional offer
        
        Returns
        -------
            bool
        """
        if dollars in self.prices:
            return True
        for price in self.prices:
            for other_price in self.prices:
                total = price + other_price
                if dollars == total:
                    return True
        return False
    
    # Returns the number of laptops that cost less than or equal to a certain price.
    # Also returns the most expensive laptop under or equal to the same certain price
    def find_best_laptop_within_budget(self, budget):
        """Prints the number of laptops that we have within a customer's given budget
        
        Returns the most expensive laptop that a customer can afford, given the customer's budget
        
        Parameters
        ----------
        budget : int
            The budget of the customer, in dollars
        
        Returns
        -------
            list
        """
        # Iterating through the binary search algorithm
        range_start = 0
        range_end = len(self.rows_by_price)
        while range_start < range_end:
            range_middle = (range_start + range_end) // 2
            price_check = self.rows_by_price[range_middle][-1]
            
            # If budget == price in list, return that laptop
            if budget == price_check:
                top_laptop = range_middle + 1
                laptops_in_budget = self.rows_by_price[:top_laptop]
                print("There are {} laptops within your budget.".format(len(laptops_in_budget)))
                return laptops_in_budget[-1]
            
            # Updating the binary search range until range convergence 
            elif budget < price_check:
                range_end = range_middle   
            else:
                range_start = range_middle + 1
                
        # After range convergence, return the most expensive laptop affordable
        
        # If budget <= price in list, then return the next lowest priced laptop
        if budget <= self.rows_by_price[range_middle][-1]:
            
            if range_middle == 0:
                return print("There are no laptops within your budget.")
                
            top_laptop = range_middle
            laptops_in_budget = self.rows_by_price[:top_laptop]
            print("There are {} laptops within your budget.".format(len(laptops_in_budget)))
            return laptops_in_budget[-1]
        
        # If budget > price in list, then return that laptop
        else:
            top_laptop = range_middle + 1
            
            if top_laptop == 1:
                laptop = self.rows_by_price[range_middle]
                print("There is 1 laptop within your budget.")
                return laptop
            
            laptops_in_budget = self.rows_by_price[:top_laptop]
            print("There are {} laptops within your budget.".format(len(laptops_in_budget)))
            return laptops_in_budget[-1]

## Instantiating the `Inventory()` Class

In [8]:
laptops = Inventory('laptops.csv')

In [9]:
# Viewing the laptops header
laptops.header

['id',
 'Company',
 'Product',
 'TypeName',
 'Inches',
 'ScreenResolution',
 'Cpu',
 'Ram',
 'Memory',
 'Gpu',
 'OpSys',
 'Weight',
 'price']

In [10]:
# Viewing the number of laptops in the dataset
len(laptops.rows)

1303

## Finding a laptop given its ID

We can search for a laptop in our list of lists, or in our `id:row` dictionary.

We have two methods that can complete this task:

* `get_laptop_from_id()` method, which has $time\ complexity$ $= O(n)$ where $n =$ the number of rows.
* `get_laptop_from_id_fast()` method, which has $time\ complexity$ $= O(1)$, because it searches a dictionary object.

In [11]:
# Using the list of lists method
laptops.get_laptop_from_id(1300)

[1300,
 'HP',
 'Stream 11-Y000na',
 'Netbook',
 '11.6',
 '1366x768',
 'Intel Celeron Dual Core N3060 1.6GHz',
 '2GB',
 '32GB Flash Storage',
 'Intel HD Graphics 400',
 'Windows 10',
 '1.17kg',
 209]

In [12]:
# Using the dictionary method
laptops.get_laptop_from_id_fast(1300)

[1300,
 'HP',
 'Stream 11-Y000na',
 'Netbook',
 '11.6',
 '1366x768',
 'Intel Celeron Dual Core N3060 1.6GHz',
 '2GB',
 '32GB Flash Storage',
 'Intel HD Graphics 400',
 'Windows 10',
 '1.17kg',
 209]

### Comparing the Time Complexity

Let's test how fast each method is.

I will test each method on a list of $10,000$ random ID values.

In [13]:
# Generating a list of 10,000 random id numbers

random_ids = [random.randint(0, 1320) for _ in range(10000)]

In [14]:
# Timing the list of lists method

total_time_no_dict = 0

for random_id in random_ids:
    
    start = time.time()
    laptops.get_laptop_from_id(random_id)
    end = time.time()
    total_time_no_dict += end - start

In [15]:
# Timing the dictionary method

total_time_dict = 0

for random_id in random_ids:
    
    start = time.time()
    laptops.get_laptop_from_id_fast(random_id)
    end = time.time()
    total_time_dict += end - start

In [16]:
# Viewing the list of lists method results
total_time_no_dict

0.3623933792114258

In [17]:
# Viewing the dictionary method results
total_time_dict

0.004407405853271484

The dictionary method is approximately $100$ $times$ faster than the list of lists method.

## Checking promotional offer amounts

We can check to see if a promotinal offer amount equals the price of one or two laptops using either of two methods:

* `check_promotion_dollars()` method, which searches through every row in the dataset and uses a nested for loop. This method has $time\ complexity$ $= O(n^2)$, where $n =$ the number of rows in the dataset.
* `check_promotion_dollars_fast()` method, which searches through the set of unique price values from the dataset, and uses a nested for loop. This method has $time\ complexity$ $= O(p^2)$ where $p =$ the number of unique price values from the dataset.

In [18]:
# This should return `True`
laptops.check_promotion_dollars(1000)

True

In [19]:
# This should return `False`
laptops.check_promotion_dollars(442)

False

In [20]:
# This should return `True`
laptops.check_promotion_dollars_fast(1000)

True

In [21]:
# This should return `False`
laptops.check_promotion_dollars_fast(442)

False

### Comparing the Time Complexity

We can test how fast each method is.

I will test each method on a list of $100$ random promotional offer values.

In [22]:
# Generating a list of 100 random promotional offer values

random_prices = [random.randint(100, 5000) for i in range(100)]

In [23]:
# Timing the datasets method

total_time_no_set = 0

for random_price in random_prices:
    
    start = time.time()
    laptops.check_promotion_dollars(random_price)
    end = time.time()
    total_time_no_set += end - start

In [24]:
# Timing the sets method

total_time_set = 0

for random_price in random_prices:
    
    start = time.time()
    laptops.check_promotion_dollars_fast(random_price)
    end = time.time()
    total_time_set += end - start

In [25]:
# Viewing the datasets method results
total_time_no_set

2.6414265632629395

In [26]:
# Viewing the sets method results
total_time_set

0.31920766830444336

The sets method is approximately $10$ $times$ faster than the datasets method.

## Finding the best laptop within a certain budget $D$

We can use our `find_best_laptop_within_budget()` method to apply a binary search algorithm that determines which laptops we can afford on a certain budget. The `find_best_laptop_within_budget()` method returns how many laptops we can afford, and the most expensive laptop available, given a certain budget:

In [27]:
# Viewing how many laptops we can afford with a $1,000 budget
# Viewing the most expensive laptop available with this budget
laptops.find_best_laptop_within_budget(1000)

There are 683 laptops within your budget.


[1058,
 'HP',
 'EliteBook 840',
 'Notebook',
 '14',
 'Full HD 1920x1080',
 'Intel Core i5 6200U 2.3GHz',
 '4GB',
 '500GB HDD',
 'Intel HD Graphics 520',
 'Windows 10',
 '1.54kg',
 1000]

In [28]:
# Viewing how many laptops we can afford with a $100 budget
laptops.find_best_laptop_within_budget(100)

There are no laptops within your budget.


##  Conclusion

In this project, we determined how much faster our algorithms run when we optimize time complexity. In one instance, our $O(1)$ algorithm ran $100\ times$ faster than our $O(n)$ algorithm! This proves invaluable at scale.

We also used a binary search algorithm to optimize the search query of which is the best laptop under a certain budget.

### Next Steps:

* Determine which laptops are available within a certain budget range.
* Determine which laptops are available given certain feature specs, like amount of RAM, Memory, etc.